In [50]:
import gridfs
from pymongo import MongoClient
from netCDF4 import Dataset 
import os
import numpy 

In [51]:
# function to read the nc file and get some field stats to use as metadata 
def get_field_stats(buf):
    ncFile = Dataset("fname", mode="r", memory=buf)
    scaled_rain = ncFile['rain_rate'][:]
    scale_factor = ncFile.variables['rain_rate'].scale_factor 
    rain = scaled_rain * scale_factor 
    
    mean = rain.mean()
    std = rain.std()
    rain_area = (rain > 0.1).sum()
    valid_area = rain.count()
    war = 100 * rain_area / valid_area 
    return mean, std, war 

In [52]:
rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
sat_name="/home/awseed/data/SAT/crr-ph/2021/20210129/20210129030000-P1S-ABOM_CRRPH-PRJ_AEA132_2000-HIMAWARI8-AHI.nc" 

In [53]:
# read the ncfile into memory 
nc_file = open(rf3_name,"rb")
buf = nc_file.read() 
nc_file.close()

In [54]:
# get the file name from the full path 
fname = os.path.basename(rf3_name)
mean, std, war = get_field_stats(buf)
print(f"mean = {mean}, std = {std}, war = {war}")

mean = 0.011121399968009088, std = 0.07087733845757768, war = 3.032964689309136


In [55]:
# # setup the database 
client = MongoClient()
rain_base = client["rain_base_fs"]
fs = gridfs.GridFSBucket(rain_base) 
file_id = fs.upload_from_stream(fname,buf,metadata={"mean":mean,"stdev":std,"WAR":war})
file_id

ObjectId('610c7c72eb33eec6974cffcf')

In [ ]:
client.close()